In [6]:
import torch
import datetime
from torch.autograd import Variable
from torchvision.models import vgg
from torchvision.datasets import coco
from torchvision import transforms

In [5]:
BASE = '/Users/ajitpunj/Documents/School/cs 349d/project/'
m = vgg.vgg11(pretrained=True)

In [7]:
ANN_PATH = 'coco/cocoapi/annotations/instances_val2017.json'
VAL_PATH = 'coco/cocoapi/images/val2017/'
test_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224), #also uses 224 cropped images
    transforms.ToTensor(),
])
def merge_coco(l):
    return torch.stack([x[0] for x in l])
c_dataset = coco.CocoDetection(root=BASE+VAL_PATH, annFile=BASE+ANN_PATH, transform=test_transform, )
eval_loader = torch.utils.data.DataLoader(c_dataset, batch_size=1, shuffle=True, collate_fn=merge_coco)

loading annotations into memory...
Done (t=0.74s)
creating index...
index created!


In [8]:
def compute_mem_size(t):
    t_size = t.size()
    n_nums = 1
    for n in t_size:
        n_nums *= n
    bit_size_map = {torch.float32 : 32, torch.float16 : 16, torch.float64 : 64, torch.uint8 : 8, torch.int8 : 8, torch.int16 : 16, torch.int32 : 32, torch.int64 : 64}
    return (n_nums * bit_size_map[t.dtype]) / 8

def vgg_measure_forward(model, x):
    print('Image size:', compute_mem_size(x))
    x = model.features(x)
    x = x.view(x.size(0), -1)
    for module in model.classifier:
        t1 = datetime.datetime.now()
        x = module(x)
        t2 = datetime.datetime.now()
        dt1 = t2 - t1
        canon_name = type(module).__name__
        print('After ' + canon_name + '(' + str(dt1.total_seconds()) + '):  ' + str(compute_mem_size(x)))
           
    return x

In [9]:
m.eval()
for batch in eval_loader:
    vgg_measure_forward(m, batch)
    break

Image size: 602112.0
After Linear(0.030582):  16384.0
After ReLU(0.000272):  16384.0
After Dropout(0.002032):  16384.0
After Linear(0.004867):  16384.0
After ReLU(0.000208):  16384.0
After Dropout(4.5e-05):  16384.0
After Linear(0.001541):  4000.0


In [10]:
transforms.ToPILImage()(sample_input)

NameError: name 'sample_input' is not defined